In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
import jieba.posseg as pseg
import jieba.analyse
import numpy as np
import tensorflow 
import random
import os
import sys
from tqdm import tqdm
import pickle

In [ ]:
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.layers import Embedding,LSTM, concatenate, Dense,Dropout,SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from scipy.stats import entropy

In [ ]:
!pip install sklearn-crfsuite
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

folder = '/drive/MyDrive/Courses/AIMAS_2022'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path =  "/content/drive/MyDrive/Courses/AIMAS_2022/sample_data.txt"

with open(file_path, 'r', encoding='utf8') as f:
    file_text = f.read().encode('utf-8').decode('utf-8-sig')

datas = file_text.split('\n\n--------------------\n\n')[:-1]

with open("/content/drive/MyDrive/Courses/AIMAS_2022/processed_data.txt", "w") as f:
    for article_id, data in enumerate(datas):
        data=data.split('\n')
        content=data[0]

        annotations=data[1:]
        row = list()
        for annot in annotations[1:]:
            annot=annot.split('\t') #annot= article_id, start_pos, end_pos, entity_text, entity_type
            row.append(annot)

        df = pd.DataFrame(row, columns=data[1].split('\t'))
        position_cols = ['start_position', 'end_position']
        df[position_cols] = df[position_cols].astype('int')

        tmp_label_list = np.array(['O'] * len(content), dtype=object)
        for i in range(len(df)):
            start, end, etype = df['start_position'][i], df['end_position'][i], df['entity_type'][i]
            # print(start, end, etype)
            tmp_label_list[start] = "B-" + str(etype)
            tmp_label_list[start+1:end] = "I-" + str(etype)

        for i, row in enumerate(zip(list(content), tmp_label_list)):
            f.write(" ".join(row) + '\n')
        
        f.write('\n')

In [ ]:
# def CRF(x_train, y_train, x_test, y_test):
#     # Doc: https://sklearn-crfsuite.readthedocs.io/en/latest/api.html#module-sklearn_crfsuite
#     # crf = sklearn_crfsuite.CRF(
#     #     algorithm='lbfgs',
#     #     c1=0.1,
#     #     c2=0.1,
#     #     max_iterations=100,
#     #     all_possible_transitions=True,
#     # )
#     # crf.fit(x_train, y_train)
    
#     y_pred = crf.predict(x_test)
#     y_pred_mar = crf.predict_marginals(x_test)

#     labels = list(crf.classes_)
#     labels.remove('O')
#     f1score = metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)
#     sorted_labels = sorted(labels,key=lambda name: (name[1:], name[0])) # group B and I results

#     return y_pred, y_pred_mar, f1score

In [ ]:
# dim = 0
# word_vecs= {}
# with open("/content/drive/MyDrive/Courses/AIMAS_2022/cna.cbow.cwe_p.tar_g.512d.0.txt") as f:
#     for line in f:
#         tokens = line.strip().split()

#         if len(tokens) == 2:
#             dim = int(tokens[1])
#             continue
    
#         word = tokens[0] 
#         vec = np.array([ float(t) for t in tokens[1:] ])
#         word_vecs[word] = vec


In [ ]:
# print(f'vocabulary_size: {len(word_vecs)}')
# print(f'word_vector_dim: {vec.shape}')

In [ ]:
def make_dataset(data_path):
    with open(data_path, 'r', encoding='utf-8') as f:
        data = f.readlines() 
    data_list, data_list_tmp = list(), list()
    article_id_list = list()
    idx = 0
    for row in data:
        data_tuple = tuple()
        if row == '\n':
            article_id_list.append(idx)
            idx+=1
            data_list.append(data_list_tmp)
            data_list_tmp = []
        else:
            row = row.strip('\n').split(' ')
            data_tuple = (row[0], row[1])
            data_list_tmp.append(data_tuple)
    if len(data_list_tmp) != 0:
        data_list.append(data_list_tmp)
    
    traindata_list, testdata_list, traindata_article_id_list, testdata_article_id_list = train_test_split(data_list,article_id_list,test_size=0.33,random_state=42)
     
    return data_list, traindata_list, testdata_list, traindata_article_id_list, testdata_article_id_list 

In [ ]:
def build_word_vectors(data_list, embedding_dict):
    embedding_list = list()

    unk_vector = np.random.rand(*(list(embedding_dict.values())[0].shape))

    for idx_list in range(len(data_list)):
        embedding_list_tmp = list()
        for idx_tuple in range(len(data_list[idx_list])):
            key = data_list[idx_list][idx_tuple][0] # token

            if key in embedding_dict:
                value = embedding_dict[key]
            else:
                value = unk_vector
            embedding_list_tmp.append(value)
        embedding_list.append(embedding_list_tmp)
        
    return embedding_list

In [ ]:
def process_labels(data_list):
    label_list = list()
    for idx_list in range(len(data_list)):
        label_list_tmp = list()
        for idx_tuple in range(len(data_list[idx_list])):
            label_list_tmp.append(data_list[idx_list][idx_tuple][1])
        label_list.append(label_list_tmp)
        
    return label_list

In [ ]:
def make_features(embed_list):
    feature_list = list()
    for idx_list in range(len(embed_list)):
        feature_list_tmp = list()
        for idx_tuple in range(len(embed_list[idx_list])):
            # feature_dict = dict()
            feature_dict=[]
            for idx_vec in range(len(embed_list[idx_list][idx_tuple])):
                feature_dict.append(embed_list[idx_list][idx_tuple][idx_vec])
                # feature_dict['dim_' + str(idx_vec+1)] = embed_list[idx_list][idx_tuple][idx_vec]
            feature_list_tmp.append(feature_dict)
        feature_list.append(feature_list_tmp)

    return feature_list

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
      
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
data_list, traindata_list, testdata_list, traindata_article_id_list, testdata_article_id_list = make_dataset("/content/drive/MyDrive/Courses/AIMAS_2022/processed_data.txt")

In [ ]:
listx=[]
listy=[]
lists=[]

longmax=0
index=0
now=0
de=0

for y in traindata_list+testdata_list:
  templong=0
  for x in y:
    templong+=1
    p="""！？｡：；～、…，"""
    if(x[0] not in p):
      listx.append(x[0])
      listy.append(x[1])
      lists.append('Sentence: '+str(now))
    if(x[0] in p):
      if(x[0]=='…'):
        if(de == 1):
          de=0
          continue
        else:
          de=1  
      now+=1
      if(templong>longmax):
        longmax=templong
        index=now
      templong=0

print(now)
print(longmax)

5649
62


In [ ]:
data = pd.DataFrame(list(zip(lists,listx,listy)),columns =['Sentence #','Word','Tag',])
data

,Sentence #,Word,Tag
0,Sentence: 0,醫,O
1,Sentence: 0,師,O
2,Sentence: 1,阿,O
3,Sentence: 1,嬤,O
4,Sentence: 1,回,O
...,...,...,...
41037,Sentence: 5649,就,O
41038,Sentence: 5649,這,O
41039,Sentence: 5649,樣,O
41040,Sentence: 5649,囉,O


In [ ]:
from collections import Counter
count_class = Counter(data["Word"])

In [ ]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words) 
print(n_words)
tags = list(set(data["Tag"].values))
n_tags = len(tags)
print(n_tags)

1225
15


In [ ]:
getter = SentenceGetter(data)
sent = getter.get_next() 
sentences = getter.sentences
len(sentences)

5647

In [ ]:
from pandas.io.parsers.base_parser import is_index_col
ss=[]
m=0
ii=0
si=0
b=35
bb=0
for s in sentences:
  ii+=1
  if(len(s)>m):
    m=len(s)
    si=ii
  if(len(s)>b):
    bb+=1
  ss.append(len(s))

ss=np.array(ss)
print(np.mean(ss))
print(np.max(ss))
print(np.min(ss))
# print(si)
print(bb)

7.267929874269524
61
1
12


In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-cc4a02ha
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-cc4a02ha


In [ ]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow_addons.text import crf_log_likelihood, crf_decode


class CRF(L.Layer):
    def __init__(self,
                 output_dim,
                 sparse_target=True,
                 **kwargs):
        """    
        Args:
            output_dim (int): the number of labels to tag each temporal input.
            sparse_target (bool): whether the the ground-truth label represented in one-hot.
        Input shape:
            (batch_size, sentence length, output_dim)
        Output shape:
            (batch_size, sentence length, output_dim)
        """
        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim) 
        self.sparse_target = sparse_target
        self.input_spec = L.InputSpec(min_ndim=3)
        self.supports_masking = False
        self.sequence_lengths = None
        self.transitions = None

    def build(self, input_shape):
        assert len(input_shape) == 3
        f_shape = tf.TensorShape(input_shape)
        input_spec = L.InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        if f_shape[-1] is None:
            raise ValueError('The last dimension of the inputs to `CRF` '
                             'should be defined. Found `None`.')
        if f_shape[-1] != self.output_dim:
            raise ValueError('The last dimension of the input shape must be equal to output'
                             ' shape. Use a linear layer if needed.')
        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, inputs, sequence_lengths=None, training=None, **kwargs):
        sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
        if sequence_lengths is not None:
            assert len(sequence_lengths.shape) == 2
            assert tf.convert_to_tensor(sequence_lengths).dtype == 'int32'
            seq_len_shape = tf.convert_to_tensor(sequence_lengths).get_shape().as_list()
            assert seq_len_shape[1] == 1
            self.sequence_lengths = K.flatten(sequence_lengths)
        else:
            self.sequence_lengths = tf.ones(tf.shape(inputs)[0], dtype=tf.int32) * (
                tf.shape(inputs)[1]
            )

        viterbi_sequence, _ = crf_decode(sequences,
                                         self.transitions,
                                         self.sequence_lengths)
        output = K.one_hot(viterbi_sequence, self.output_dim)
        return K.in_train_phase(sequences, output)

    @property
    def loss(self):
        def crf_loss(y_true, y_pred):
            y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
            log_likelihood, self.transitions = crf_log_likelihood(
                y_pred,
                tf.cast(K.argmax(y_true), dtype=tf.int32) if self.sparse_target else y_true,
                self.sequence_lengths,
                transition_params=self.transitions,
            )
            return tf.reduce_mean(-log_likelihood)
        return crf_loss

    @property
    def accuracy(self):
        def viterbi_accuracy(y_true, y_pred):
            mask = K.cast(
                K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            if self.sparse_target:
                y_true = K.argmax(y_true, 2)
            y_pred = K.cast(y_pred, 'int32')
            y_true = K.cast(y_true, 'int32')
            corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
            return K.sum(corrects * mask) / K.sum(mask)
        return viterbi_accuracy

    def compute_output_shape(self, input_shape):
        tf.TensorShape(input_shape).assert_has_rank(3)
        return input_shape[:2] + (self.output_dim,)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'sparse_target': self.sparse_target,
            'supports_masking': self.supports_masking,
            'transitions': K.eval(self.transitions)
        }
        base_config = super(CRF, self).get_config()

In [ ]:
tags

['B-time',
 'I-time',
 'I-location',
 'I-med_exam',
 'B-location',
 'I-name',
 'B-profession',
 'B-organization',
 'I-money',
 'I-organization',
 'B-med_exam',
 'B-money',
 'O',
 'B-name',
 'I-profession']

In [ ]:
max_len = 37
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)
y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.33,random_state=2023)

In [ ]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

num_labels = 15
crf = CRF(num_labels, sparse_target=True)
adam = tensorflow.keras.optimizers.Adam(lr=0.005)
model = Sequential()
model.add(Embedding(input_dim=n_words+1, output_dim=200, input_length=max_len))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(TimeDistributed(Dense(n_tags, activation="relu")))
model.add(crf)
model.compile('adam', loss=crf.loss, metrics=[crf.accuracy])
model.summary()

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 37, 200)           245200    
                                                                 
 dropout_2 (Dropout)         (None, 37, 200)           0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 37, 256)          336896    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 37, 15)           3855      
 tributed)                                                       
                                                                 
 crf_3 (CRF)                 (None, 37, 15)            225       
                                                                 
Total params: 586,176
Trainable params: 586,176
Non-tr

In [ ]:
x_train=np.array(X_tr)
y_train=np.array(y_tr)
x_train = np.asarray(x_train,dtype='float32')
y_train = np.asarray(y_train,dtype='float32')
model.fit(x_train, y_train, batch_size=16, epochs=50, verbose=1)

Epoch 1/50
237/237 [==============================] - 47s 159ms/step - loss: 4.0048 - viterbi_accuracy: 0.9850
Epoch 2/50
237/237 [==============================] - 27s 116ms/step - loss: 0.8474 - viterbi_accuracy: 0.9946
Epoch 3/50
237/237 [==============================] - 28s 116ms/step - loss: 0.6964 - viterbi_accuracy: 0.9951
Epoch 4/50
237/237 [==============================] - 28s 119ms/step - loss: 0.5701 - viterbi_accuracy: 0.9958
Epoch 5/50
237/237 [==============================] - 31s 129ms/step - loss: 0.4796 - viterbi_accuracy: 0.9963
Epoch 6/50
237/237 [==============================] - 28s 119ms/step - loss: 0.4265 - viterbi_accuracy: 0.9965
Epoch 7/50
237/237 [==============================] - 28s 120ms/step - loss: 0.3961 - viterbi_accuracy: 0.9966
Epoch 8/50
237/237 [==============================] - 28s 117ms/step - loss: 0.3536 - viterbi_accuracy: 0.9968
Epoch 9/50
237/237 [==============================] - 28s 117ms/step - loss: 0.3205 - viterbi_accuracy: 0.9970
E

In [ ]:
test_pred = model.predict(X_te, verbose=1)

pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

labels = list(tags)
labels.remove('O')

f1score = metrics.flat_f1_score(test_labels, pred_labels, average='weighted', labels=labels)
f1score

59/59 [==============================] - 4s 53ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.6997282722069188

In [ ]:
# for i in range(0,50):
#   # i = 50
#   p = model.predict(np.array([X_te[i]]))
#   p = np.argmax(p, axis=-1)
#   true = np.argmax(y_te[i], -1)
#   print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
#   print(30 * "=")
#   for w, t, pred in zip(X_te[i], true, p[0]):
#       if w != 0:
#           print("{:15}: {:5} {}".format(words[w-1], tags[t], tags[pred]))

In [ ]:
# n_tags=15
# inputs = Input(shape=(None,), dtype='int32')
# output = Embedding(input_dim=n_words+1, output_dim=200, input_length=max_len)(inputs)
# output = Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.1))(output)
# output = TimeDistributed(Dense(n_tags, activation="relu"))(output)
# crf = tfa.layers.CRF(n_tags)
# decoded_sequence, potentials, sequence_length, chain_kernel = crf(output) 
# model = Model(inputs, potentials)
# model.compile(optimizer="adam", loss=crf.add_loss, metrics=['accuracy'])

# model = Sequential()
# model.add(Embedding(input_dim=n_words+1, output_dim=200, input_length=max_len))
# model.add(Dropout(0.5))
# model.add(Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.1)))
# model.add(TimeDistributed(Dense(n_tags, activation="relu")))
# crf_layer = tfa.layers.CRF(n_tags)
# model.add(crf_layer)

In [ ]:
# pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# class Data_set:
#     def __init__(self,data_path,labels):
#         with open(data_path,"rb") as f:
#             self.data = f.read().decode("utf-8")
#         self.process_data = self.process_data()
#         self.labels = labels
#     def process_data(self):
#         train_data =self.data.split("\n\n")
#         train_data = [token.split("\n") for token in train_data]
#         train_data = [[j.split() for j in i ] for i in train_data]
#         train_data.pop()
#         return train_data
    
#     def save_vocab(self,save_path):
#         all_char = [ char[0] for sen in self.process_data for char in sen]
#         chars = set(all_char)
#         word2id = {char:id_+1 for id_,char in enumerate(chars)}
#         word2id["unk"] = 0
#         with open (save_path,"wb") as f:
#             pickle.dump(word2id,f)
#         return word2id    
    
#     def generate_data(self,vocab,maxlen):
#         char_data_sen = [[token[0] for token in i ] for i in self.process_data]
#         label_sen = [[token[1] for token in i ] for i in self.process_data]
#         sen2id = [[ vocab.get(char,0) for char in sen] for sen in char_data_sen]
#         label2id = {label:id_ for id_,label in enumerate(self.labels)}
#         lab_sen2id = [[label2id.get(lab,0) for lab in sen] for sen in label_sen]
#         sen_pad = pad_sequences(sen2id,maxlen)
#         lab_pad = pad_sequences(lab_sen2id,maxlen,value=-1)
#         lab_pad = np.expand_dims(lab_pad, 2)
#         return sen_pad ,lab_pad

In [ ]:
# !pip install tensorflow_addons
# from tensorflow_addons.layer import CRF
# !pip install tf2crf

In [ ]:
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
# from keras_contrib.layers import CRF
# from keras_contrib.losses import crf_loss
# from keras_contrib.metrics import crf_accuracy

# class Ner:
#     def __init__(self,vocab,labels_category,Embedding_dim=200):
#         self.Embedding_dim = Embedding_dim
#         self.vocab = vocab
#         self.labels_category = labels_category
#         self.model = self.build_model()


#     def build_model(self):
#         inputs = Input(shape=(None,), dtype='int32')
#         output = Embedding(100, 40, trainable=True, mask_zero=False)(inputs)
#         output = Bidirectional(GRU(64, return_sequences=True))(output)
#         output = Dense(9, activation=None)(output)
#         crf = CRF(dtype='float32')
#         output = crf(output)
#         model = Model(inputs, output)
#         model.compile(loss=crf.loss, optimizer='adam', metrics=[crf.accuracy])
#         # n_words=len(x_train)
#         # max_len=75
#         # n_tags=11
#         # model = Sequential()
#         # model.add(Embedding(input_dim=n_words+1, output_dim=200, input_length=max_len))
#         # model.add(Dropout(0.5))
#         # model.add(Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.1)))
#         # model.add(TimeDistributed(Dense(n_tags, activation="relu")))
#         # crf_layer = CRF(n_tags)
#         # model.add(crf_layer)
#         # model.summary()
#         # model.compile(optimizer='adam', loss=crf_layer.loss_function, metrics=[crf_layer.accuracy])
#         return model

#     def train(self,data,label,EPOCHS):
#         self.model.fit(data,label,batch_size=16,epochs=EPOCHS)
#         self.model.save('crf.h5')
        
#     def predict(self,model_path,data,maxlen):
#         model =self.model
#         char2id = [self.vocab.get(i) for i in data]
#         pad_num = maxlen - len(char2id)
#         input_data = pad_sequences([char2id],maxlen)
#         model.load_weights(model_path)
#         result = model.predict(input_data)[0][-len(data):]
#         result_label = [np.argmax(i) for i in result]
#         return result_label

In [ ]:
# from tf2crf import CRF
# from tensorflow.keras.layers import Input, Embedding, Bidirectional, GRU, Dense
# from tensorflow.keras.models import Model

# inputs = Input(shape=(None,), dtype='int32')
# output = Embedding(100, 40, trainable=True, mask_zero=True)(inputs)
# output = Bidirectional(GRU(64, return_sequences=True))(output)
# output = Dense(9, activation=None)(output)
# crf = CRF(dtype='float32')
# output = crf(output)
# model = Model(inputs, output)
# model.compile(loss=crf.loss, optimizer='adam', metrics=[crf.accuracy])

# x = [[5, 2, 3] * 3] * 10
# y = [[1, 2, 3] * 3] * 10

# model.fit(x=x, y=y, epochs=2, batch_size=2)
# model.save('model')

In [ ]:
# tags = ['B-location','I-location','B-time','I-time','B-med_exam','I-med_exam','B-name','I-name','B-money','I-money','O','B-profession',
#     'I-profession','B-organization','I-organization']
# data = Data_set("/content/drive/MyDrive/Courses/AIMAS_2022/processed_data.txt",tags)
# vocab = data.save_vocab("vocab.pk")
# sentence,sen_tags= data.generate_data(vocab,75)

In [ ]:
# ner = Ner(vocab,tags)

In [ ]:
# ner.train(sentence,sen_tags,1)

In [ ]:
# trainembed_list = build_word_vectors(traindata_list, word_vecs)
# testembed_list = build_word_vectors(testdata_list, word_vecs)

# x_train = make_features(trainembed_list)
# y_train = process_labels(traindata_list)

# x_test = make_features(testembed_list)
# y_test = process_labels(testdata_list)

# x=[]
# for xt in x_train:
#   for xxt in xt:
#     x.append(xxt)
# y=[]
# for xt in y_train:
#   for xxt in xt:
#     y.append(xxt)

# x_train=x
# y_train=y 


# x=[]
# for xt in x_test:
#   for xxt in xt:
#     x.append(xxt)
# y=[]
# for xt in y_test:
#   for xxt in xt:
#     y.append(xxt)

# x_test=x
# y_test=y 

# print(len(x_train))
# print(len(y_train))
# model = Sequential()

# MAX_NUM_WORDS = 360000
# MAX_SEQUENCE_LENGTH = 1000000
# NUM_CLASSES = 11
# NUM_EMBEDDING_DIM = 512
# NUM_LSTM_UNITS =512
# # import pickle
# import numpy as np
# from keras import Sequential
# from keras_contrib.layers import CRF
# import pickle
# from keras.layers import Embedding ,Bidirectional,LSTM

# model = Sequential()

# model.add(Embedding(30000,64))  # Random embedding

# model.add(Bidirectional(LSTM(100, return_sequences=True)))

# # # crf = CRF(11, sparse_target=True)
# # # model.add(crf)
# # model.summary()
# # model.compile('adam', loss=crf.loss_function, metrics=[crf.accuracy])

# # model.add(Embedding(MAX_NUM_WORDS, NUM_EMBEDDING_DIM,trainable=False))

# # model.add(LSTM(NUM_LSTM_UNITS))

# model.add(Dense(units=NUM_CLASSES,activation='softmax'))

# print(model.summary())

# model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

# label_to_index = {
#     'B-location':0,
#     'I-location':1,
#     'B-time':2,
#     'I-time':3,
#     'B-med_exam':4,
#     'I-med_exam':5,
#     'B-name':6,
#     'I-name':7,
#     'B-money':8,
#     'I-money':9,
#     'O':10,
#     # 'B-profession':10,
#     # 'I-profession':10,
#     # 'B-organization':10,
#     # 'I-organization':10,

#   }

# BATCH_SIZE = 64
# NUM_EPOCHS = 5

# y_train= pd.DataFrame(y_train,columns =['Names'])
# y_train = y_train.Names.apply(lambda x: label_to_index[x])
# y_train =tensorflow.keras.utils.to_categorical(y_train)
# y_train = np.asarray(y_train)


# x_train = np.asarray(x_train)

# print(len(x_train))
# print(len(y_train))

# # y_test=pd.DataFrame(y_test,columns =['Names'])
# # y_test = y_test.Names.apply(lambda x: label_to_index[x])
# # store=y_test
# # y_test =tensorflow.keras.utils.to_categorical(y_test)

# # x_test = np.asarray(x_test)
# # y_test = np.asarray(y_test)

# history = model.fit(
#     x=x_train,
#     y=y_train,
#     batch_size=BATCH_SIZE,
#     epochs=NUM_EPOCHS,
#     # validation_data=(x_test,y_test),
#     shuffle=False,
#     verbose=1
# ) 

# # y_test=store

In [ ]:
# y_test=pd.DataFrame(y_test[0],columns =['Names'])
# y_test = y_test.Names.apply(lambda x: label_to_index[x])
# x_test = np.asarray(x_test[0])
# y_pred = model.predict(x_test)
# y_pred=np.argmax(y_pred,axis=1)

In [ ]:
# from sklearn.metrics import f1_score
# labels = [0,1,2,3,4,5,6,7,8,9]
# f1_score(y_test, y_pred,average='weighted',labels=labels)

In [ ]:
# index_to_label = {v: k for k, v in label_to_index.items()}

# output = [index_to_label[idx] for idx in np.argmax(y_pred, axis=1)]

In [ ]:
# output="article_id\tstart_position\tend_position\tentity_text\tentity_type\n"
# for test_id in range(len(y_pred)):
#     pos=0
#     start_pos=None
#     end_pos=None
#     entity_text=None
#     entity_type=None
#     for pred_id in range(len(y_pred[test_id])):
#         if y_pred[test_id][pred_id][0]=='B':
#             start_pos=pos
#             entity_type=y_pred[test_id][pred_id][2:]
#         elif start_pos is not None and y_pred[test_id][pred_id][0]=='I' and y_pred[test_id][pred_id+1][0]=='O':
#             end_pos=pos
#             entity_text=''.join([testdata_list[test_id][position][0] for position in range(start_pos,end_pos+1)])
#             line=str(testdata_article_id_list[test_id])+'\t'+str(start_pos)+'\t'+str(end_pos+1)+'\t'+entity_text+'\t'+entity_type
#             output+=line+'\n'
#         pos+=1

In [ ]:
# output_path='output.tsv'
# with open(output_path,'w',encoding='utf-8') as f:
#     f.write(output)

In [ ]:
# print(output)

-----------------------------------------------------